# for each chunk: time each and match times where possible (streamline with APIs?)
## - demucs separation - script will allow direct from file
### - check for stereo: is stereo in output here?
ffprobe rome.m4a 2>&1 | grep -A1 Duration
### - split into two
ffmpeg -i stereo.wav -map_channel 0.0.0 left.wav -map_channel 0.0.1 right.wav
### - funnel each into demucs
### - move from there up to analysis location
## - load split files
## - librosa: 
### - update color scheme
### - circles: stack by lowest amp as total amp area, to highest amp as center
### - possible to separate out by fundamental frequency?
### - stereo effects? subtract wave from each channel, and difference = x value
## - save frames > send directly to video
## - make movie
## - add audio 

In [ ]:
import sys
import pandas as pd
import numpy as np
import librosa, sklearn
import matplotlib.pyplot as plt
import librosa.display
import IPython.display
from datetime import datetime
%matplotlib inline
import cv2
import os
from tqdm import tqdm
import seaborn as sns
from streamlit import caching
plt.rcParams['figure.figsize'] = [16, 12]

#add white to colorlist for drums
colorList = sns.color_palette('hsv', 12)
colorList+=[(1,1,1)]
#balance between low number for precision, high number for stability of pitch/amp calc
hop_length = 2205
#load all files, 
media_name = 'rome.m4a'
filename = media_name.split('.')[0]
print('loading')
channels = {}

for i, source in enumerate(['vocals','drums','other','bass']):
    channels['filename'] = filename.split('.')[0]
    channels[source] = {}
#     channels[source]['combined'] = {}
    channels[source][0] = {}
    channels[source][1] = {}
#     channels[source]['combined']['audio'], sr = librosa.load(f"demucs/separated/demucs/{filename}/{source}/{source}.wav", sr=None)
#         split here?
    os.system(f"ffmpeg -i {source}/{source}.wav -map_channel 0.0.0 {source}/left.wav -map_channel 0.0.1 {source}/right.wav")
    channels[source][0]['audio'], sr = librosa.load(f"demucs/separated/demucs/{filename}/{source}/left.wav", sr=None)
    channels[source][1]['audio'], sr = librosa.load(f"demucs/separated/demucs/{filename}/{source}/right.wav", sr=None)
#     channels[source]['stft'] = librosa.stft(channels[source]['audio'])



In [ ]:
# frames = range(len(channels[channel]['audio']))
# t = librosa.frames_to_time(frames[0:1000*512])
# for channel in channels.keys():
#     plt.plot(t,np.abs(channels[channel]['audio'][0:1000*512]))
# plt.legend(channels.keys())

In [ ]:
# testright = channels['other'][1]['audio']*-1+channels['other'][0]['audio']
testleft = channels['other'][0]['audio']*-1+channels['other'][1]['audio']
IPython.display.Audio(data=channels['other'][0]['audio'], rate=sr, autoplay = True)


In [ ]:
IPython.display.Audio(data=channels['other'][0]['audio'], rate=sr, autoplay = True)


In [ ]:
samprange = range(0, sr*25)
plt.plot(samprange, 
#          channels['other'][0]['audio'][samprange]-channels['other'][1]['audio'][samprange], 
#          channels['other'][1]['audio'][samprange]-channels['other'][0]['audio'][samprange], 
         channels['other'][0]['audio'][samprange], 
         channels['other'][1]['audio'][samprange])
plt.legend(['l-r','r-l','left','right'])

In [ ]:
for i, source in enumerate([k for k in channels.keys() if k not in ['filename','drums']]):
    print(source)

In [ ]:
for source in channels.keys():
    if source != 'filename':
        for channel in [0,1]:
            if source != 'drums':
                #pitch is defaulting to F across the board on Rome. is that the case for OW too?
                chromagram = librosa.feature.chroma_stft(channels[source][channel]['audio'], sr=sr, hop_length=hop_length)
                channels[source][channel]['pitch'] = chromagram
            else:
                channels[source][channel]['pitch'] = np.repeat(12, int(np.floor(len(channels[source][channel]['audio'])/hop_length)))
                
            channels[source][channel]['amp'] = np.array([np.mean(abs(channels[source][channel]['audio'][(s*hop_length):(s*hop_length+hop_length)])) 
                                            for s in range(int(np.floor(len(channels[source][channel]['audio'])/hop_length))-1)])
        

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation


def plotimages(s, dataDict, colorList):
    # take maximum-amplitude pitch for the sample, normalize sample ampl by maximum ampl (or by "other"?)
    # for now, use pitch from channel with larger amp. 
    # how to handle two-channel bass better? revert back to circles?
    channel = np.argmax([abs(dataDict['bass'][0]['amp'][s]),abs(dataDict['bass'][1]['amp'][s])])
    bgcolor = [c*0.5*(abs(dataDict['bass'][channel]['amp'][s])/
                      abs(max(dataDict['bass'][channel]['amp']))) for c in 
               colorList[np.argmax(dataDict['bass'][channel]['pitch'][:,s])]] 
                                   
    plt.rcParams['axes.facecolor'] = bgcolor
    plt.rcParams['savefig.facecolor'] = bgcolor
    
    fig, ax = plt.subplots(1,1)
    for channel in [0,1]:
        #drums
        plt.plot(1+channel,1.5, 'o',
                 markerfacecolor = colorList[dataDict['drums'][channel]['pitch'][s]], 
                 markeredgecolor = colorList[dataDict['drums'][channel]['pitch'][s]], 
                 markersize = dataDict['drums'][channel]['amp'][s]*100)

        #grab pitches in reverse order of strength, only using top few
        topchr = np.argsort(dataDict['vocals'][channel]['pitch'][:,s])
        for pitch_rank in range(-1, 1):
            plt.plot(.5+channel*2,0.75, 'o',
                     markerfacecolor = colorList[topchr[pitch_rank]], 
                     markeredgecolor = colorList[topchr[pitch_rank]],
                     markersize = 250*dataDict['vocals'][channel]['amp'][s]*sum(dataDict['vocals'][channel]['pitch'][topchr[range(pitch_rank,1)],s]))

        topchr = np.argsort(dataDict['other'][channel]['pitch'][:,s])
        for pitch_rank in range(-1, 1):
            plt.plot(0.5+channel*2,2.25, 'o',
                     markerfacecolor = colorList[topchr[pitch_rank]], 
                     markeredgecolor = colorList[topchr[pitch_rank]],
                     markersize = 250*dataDict['other'][channel]['amp'][s]*sum(dataDict['other'][channel]['pitch'][topchr[range(pitch_rank,1)],s]))
    plt.xlim(0,3)
    plt.ylim(0,3)
    plt.axis('off')
    digits = len(str(len(dataDict['vocals'][0]['amp'])))
    plt.savefig(f"{dataDict['filename']}_frames2/{s:0{digits}d}.png", 
                facecolor=ax.get_facecolor(), edgecolor='none')
    plt.close()
#     canvas = plt.gca().figure.canvas
#     canvas.draw()
#     data = np.frombuffer(canvas.tostring_rgb(), dtype=np.uint8)
#     image = data.reshape(canvas.get_width_height()[::-1] + (3,))
# #     #alternative?
# #      # redraw the canvas
#     fig.canvas.draw()
#     # convert canvas to image
#     img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
#     img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
# # then outside
#     # img is rgb, convert to opencv's default bgr
# #     img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
# #     cv2.imshow("plot",img)

#     return [img, image]


In [ ]:
digits = len(str(len(channels['vocals'][0]['amp'])))

startt = datetime.now()
if f"{filename}_frames2" not in os.listdir():
    os.mkdir(f"{filename}_frames2")

start_frame = len(sorted(os.listdir(f'{filename}_frames2/')))-1

while start_frame < 2000:
    print(f"{start_frame}")
    for i in tqdm(range(start_frame, min(start_frame+1000, len(channels['vocals'][0]['amp'])))):
        plotimages(i, dataDict = channels, colorList = colorList)


    image = cv2.imread(f"{filename}_frames/{os.listdir(f'{filename}_frames/')[0]}")
    height, width, layers = image.shape
    size = (width,height)

    if f"{filename}.avi" in os.listdir():
        os.remove(f"{filename}.avi")

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    os.system(f"rm -f {filename}.avi")
    video = cv2.VideoWriter(f"{filename}.avi", 
                            fourcc, 
                            sr/hop_length, 
                            size)
    for i in tqdm(range(len(sorted(os.listdir(f'{filename}_frames'))))):
        image = cv2.imread(f'{filename}_frames/{i:0{digits}d}.png')
        video.write(image.astype('uint8'))

    video.release()
    cv2.destroyAllWindows()
    print(datetime.now()-startt)
    if f"{filename}_waud.avi" in os.listdir():
        os.remove(f"{filename}_waud.avi")
    os.system(f"ffmpeg -i {filename}.avi -i {media_name} -map 0 -map 1:a -c:v copy -shortest {filename}_waud.avi")

    caching.clear_cache()
    
#     print(datetime.now()-startt)
#     clip_1 = VideoFileClip(f"{filename}.avi")
#     clip_2 = VideoFileClip(f"{filename}_tmp.avi")
#     final_clip = concatenate_videoclips([clip_1,clip_2])
#     final_clip.write_videofile(f"{filename}.avi")
# this downsamples the video, it seems? ugh.
# os.system("ffmpeg -i rome.avi -i rome_waud.avi -filter_complex '[0:v] [0:a:0] [1:v] [1:a:0] concat=n=2:v=1:a=1 [v] [a]' -map '[v]' -map '[a]' rome_concat.avi")

    start_frame = len(sorted(os.listdir(f'{filename}_frames/')))
    

In [ ]:
f"ffmpeg -i {filename}.avi -i {media_name} -map 0 -map 1:a -c:v copy -shortest {filename}_waud.avi"


In [ ]:
filename = 'rome'
media_name = 'rome.m4a'
f"ffmpeg -i {filename}.avi -i {media_name} -map 0 -map 1:a -c:v copy -shortest {filename}_waud.avi"

In [ ]:
i

In [ ]:
# images = []
# for img in sorted(os.listdir(image_folder)):
#     if img.endswith(".jpg"):
#         for i in range(100):
#             image_files += [f"{image_folder}/{img}"]


In [ ]:
from streamlit import caching
caching.clear_cache()

In [ ]:
import soundfile as sf
# y, srdown = librosa.load(f"catempire_oscarwilde.mp3", sr = 4300)
sf.write('catempire_oscarwilde_47.wav', ow_47, sr)         

In [ ]:
import os
import moviepy
from moviepy.audio.io.AudioFileClip import AudioFileClip
from moviepy.audio.AudioClip import CompositeAudioClip
import moviepy.video.io.ImageSequenceClip
from datetime import datetime
image_folder='catempire_oscarwilde_frames/'
srdown = sr/hop_length
image_files = []

for img in sorted(os.listdir(image_folder)):
    if img.endswith(".jpg"):
        for i in range(100):
            image_files += [f"{image_folder}/{img}"]
startt = datetime.now()
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=srdown) #pass in list of images instead?
print(datetime.now()-startt)
# audioclip = AudioFileClip("catempire_oscarwilde_47.wav", fps = sr)

# # new_audioclip = CompositeAudioClip([audioclip])
# clip.audio = audioclip
clip.write_videofile("oscarwilde100_2.mp4", fps = srdown, threads=50)

In [ ]:
ow_47 = librosa.load('catempire_oscarwilde.mp3', duration=47)
IPython.display.Audio(data=ow_47, rate=sr, autoplay = True)


In [ ]:
chromagram = librosa.feature.chroma_stft(channels['vocals']['audio'], sr=sr, hop_length=512)
chromagram.shape
librosa.display.specshow(chromagram[:,0:1000], x_axis='time', y_axis='chroma')


In [ ]:
chromagram = librosa.feature.chroma_stft(channels['other']['audio'], sr=sr, hop_length=512)
librosa.display.specshow(chromagram[:,0:1000], x_axis='time', y_axis='chroma')


In [ ]:
IPython.display.Audio(data=channels[channel]['audio'], rate=sr/1.5, autoplay = True)


In [ ]:
y, sr = librosa.load('catempire_oscarwilde.mp3')
D = librosa.stft(y)

# Pre-compute a global reference power from the input spectrum
rp = np.max(np.abs(D))


In [ ]:
%matplotlib inline
D_harmonic, D_percussive = librosa.decompose.hpss(D, kernel_size = [1000,1000])

fig, ax = plt.subplots(nrows=3, sharex=True, sharey=True)

img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(D), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[0])
ax[0].set(title='Full spectrogram')
ax[0].label_outer()

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[1])
ax[1].set(title='Harmonic spectrogram')
ax[1].label_outer()

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[2])
ax[2].set(title='Percussive spectrogram')
fig.colorbar(img, ax=ax)

In [ ]:
D_harmonic, D_percussive = librosa.decompose.hpss(D, kernel_size = [15,1000])

# IPython.display.Audio(data=librosa.istft(D_harmonic), rate=sr, autoplay = True)
# IPython.display.Audio(data=librosa.istft(D_percussive), rate=sr, autoplay = True)


In [ ]:
# yfb, sr = librosa.load('catempire_oscarwilde/drums.wav')
# Dfb = librosa.stft(yfb)

# Pre-compute a global reference power from the input spectrum
rpfb = np.max(np.abs(Dfb))


In [ ]:
plt.rcParams['figure.figsize'] = [16, 12]
rp = np.max(np.abs(D))

fig, ax = plt.subplots(nrows=3, sharex=True, sharey=True)

img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(D), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[0])
ax[0].set(title='Full spectrogram')
ax[0].label_outer()

librosa.display.specshow(librosa.amplitude_to_db(np.abs(Dfb), ref=rpfb),
                         y_axis='log', x_axis='time', ax=ax[1])
ax[1].set(title='Harmonic spectrogram')
ax[1].label_outer()

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[2])
ax[2].set(title='Percussive spectrogram')
fig.colorbar(img, ax=ax)

In [ ]:
load_dur = 90
y, sr = librosa.load('catempire_oscarwilde.mp3', duration = load_dur)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
zcr = librosa.feature.zero_crossing_rate(y)[0]
mfcc = librosa.feature.mfcc(y=y, sr=sr)
D = librosa.stft(y)
rf = np.max(np.abs(D))

plt.rcParams['figure.figsize'] = [16, 12]

fig, ax = plt.subplots(5,1)
librosa.display.specshow(librosa.amplitude_to_db(D, ref=rp),y_axis='log', x_axis='time', ax=ax[0])
ax[0].set(title='Spectogram')

def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)#Plotting the Spectral Centroid along the waveform
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)[0]

frames = range(len(spec_cent))
t = librosa.frames_to_time(frames)
# librosa.display.waveplot(y, sr=sr, alpha=0.4)
ax[1].plot(t, spec_cent, color='r')
ax[1].set(title='spectral centroid')
ax[1].set_xlim([0,load_dur])

mfccs = sklearn.preprocessing.scale(mfcc, axis=1)
librosa.display.specshow(mfccs, sr=sr, x_axis='time', ax=ax[2])
ax[2].set(title='MFCC')

hop_length = 28
chromagram = librosa.feature.chroma_stft(y, sr=sr, hop_length=hop_length)
librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', ax = ax[3])
ax[3].set(title = 'chromagram')

ax[4].plot(spec_bw, color='r')
ax[4].set(title='spectral bandwidth')
ax[4].set_xlim([0,load_dur])


In [ ]:
IPython.display.Audio(data=y, rate=sr/1.5, autoplay = True)


In [ ]:
y_offset, sr = librosa.load('catempire_oscarwilde.mp3', duration=60, offset = 60)
D = librosa.stft(y_offset)
rp = np.max(np.abs(D))

#higher percussive kernel_size better for isolating
# y_h1515, y_p1515 = librosa.effects.hpss(y_offset, kernel_size = [15,1000]) 
# D_H1515, D_P1515 = librosa.decompose.hpss(D, kernel_size = [15,1000])
# IPython.display.Audio(data=y_h1515, rate=sr, autoplay = True)

In [ ]:
y_h155, y_p155 = librosa.effects.hpss(y, kernel_size = [15,5])
D_H155, D_P155 = librosa.decompose.hpss(D, kernel_size = [15,5])

fig, ax = plt.subplots(nrows=4, sharex=True, sharey=True)
dp = {}
dh = {}
for i,ks in enumerate([5,10,100,1000]):
    dh[i], dp[i] = librosa.decompose.hpss(D, kernel_size = [15,ks])

    librosa.display.specshow(librosa.amplitude_to_db(np.abs(dp[i][:, 0:250]), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[i])
    ax[i].set(title=f'{ks}')

# IPython.display.Audio(data=y_p1515[0:sr*10], rate=sr, autoplay = True)


In [ ]:
# test = datetime.datetime.now()/
rp = np.max(np.abs(D))


In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, sharex=True, sharey=True)
dp = {}
dh = {}
yp = {}
yh = {}
start = datetime.datetime.now()
for i,ks in enumerate([5,10,100,1000]):
    dh[i], dp[i] = librosa.decompose.hpss(D, kernel_size = [ks,1000])
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(dh[i][:, 0:250]), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[i,0])
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(dp[i][:, 0:250]), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[i,1])
    ax[i,0].set(title=f'{ks}, harmonic')
    ax[i,1].set(title=f'{ks}, Percussive')
    yh[i] = librosa.istft(dh[i])
    yp[i] = librosa.istft(dp[i])
    print(datetime.datetime.now()-start)

# IPython.display.Audio(data=yp[0], rate=sr, autoplay = True)


In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, sharex=True, sharey=True)
dp3 = {}
dh3 = {}
yp3 = {}
yh3 = {}
start = datetime.datetime.now()
for i,ks in enumerate([5,10,100,1000]):
    dh3[i], dp3[i] = librosa.decompose.hpss(D, kernel_size = [ks,31])
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(dh3[i][:, 0:250]), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[i,0])
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(dp3[i][:, 0:250]), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[i,1])
    ax[i,0].set(title=f'{ks}, harmonic')
    ax[i,1].set(title=f'{ks}, Percussive')
    yh3[i] = librosa.istft(dh3[i])
    yp3[i] = librosa.istft(dp3[i])
    print(datetime.datetime.now()-start)

# IPython.display.Audio(data=yp[0], rate=sr, autoplay = True)


In [ ]:
IPython.display.Audio(data=yp[1], rate=sr, autoplay = True)


In [ ]:
yb, srb = librosa.load(librosa.ex('vibeace'), duration = 60)
Db = librosa.stft(yb)
rpb = np.max(np.abs(Db))
fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True)

librosa.display.specshow(librosa.amplitude_to_db(np.abs(Db), ref=rpb),
                         y_axis='log', x_axis='time', ax=ax)
IPython.display.Audio(data=yb, rate=srb, autoplay = True)


In [ ]:
# Let's compute separations for a few different margins and compare the results below
# D_harmonic2, D_percussive2 = librosa.decompose.hpss(D, margin=2)
# D_harmonic4, D_percussive4 = librosa.decompose.hpss(D, margin=4)
# D_harmonic8, D_percussive8 = librosa.decompose.hpss(D, margin=8)
# D_harmonic16, D_percussive16 = librosa.decompose.hpss(D, margin=16)

fig, ax = plt.subplots(nrows=4, sharex=True, sharey=True)

img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic2), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[0])

img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic4), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[1])

img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic8), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[2])

img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic16), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[3])

In [ ]:
y_harmonic2, y_percussive2 = librosa.effects.hpss(y, margin=2)
IPython.display.Audio(data=y_harmonic2, rate=sr)

y_harmonic4, y_percussive4 = librosa.effects.hpss(y, margin=4)
IPython.display.Audio(data=y_harmonic4, rate=sr)

y_harmonic8, y_percussive8 = librosa.effects.hpss(y, margin=8)
IPython.display.Audio(data=y_harmonic8, rate=sr)

y_harmonic16, y_percussive16 = librosa.effects.hpss(y, margin=16)
IPython.display.Audio(data=y_harmonic16, rate=sr)




In [ ]:
IPython.display.Audio(data=y_h, rate=sr, autoplay = True)

In [ ]:
# How about something more advanced?  Let's decompose a spectrogram with NMF, and then resynthesize an individual component
D = librosa.stft(y)
rp = np.max(np.abs(D))

# Separate the magnitude and phase
S, phase = librosa.magphase(D_harmonic)

# Decompose by nmf
components, activations = librosa.decompose.decompose(S, n_components=4, sort=True)
# Resynthesize.  How about we isolate components?
fig, ax = plt.subplots(nrows=len(activations), sharex=True, sharey=True)

y_k = list(range(len(activations)))
for k in range(len(activations)):
    
    # Reconstruct a spectrogram by the outer product of component k and its activation
    D_k = np.multiply.outer(components[:, k], activations[k])

    # invert the stft after putting the phase back in
    y_k[k] = librosa.istft(D_k * phase)

    # And playback
    print('Component #{}'.format(k))

    IPython.display.Audio(data=y_k[k], rate=sr)
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_k), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[k])

    

In [ ]:
IPython.display.Audio(data=y_k[2], rate=sr, autoplay=True)

In [ ]:
IPython.display.Audio(data=y, rate=sr, autoplay=True)